In [5]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl


class MNISTModel(pl.LightningModule):

    def __init__(self):
        super(MNISTModel, self).__init__()
        # not the best model...
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        # called with self(x)
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        # REQUIRED
        return DataLoader(MNIST(os.getcwd(), train=True, download=True,
                                transform=transforms.ToTensor()), batch_size=32)

    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=True, download=True,
                                transform=transforms.ToTensor()), batch_size=32)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=False, download=True,
                                transform=transforms.ToTensor()), batch_size=32)
    
model = MNISTModel()
train_dl = model.train_dataloader()

for x_train in train_dl:
    model.training_step(x_train, 0)
    break

In [3]:
mnist_model = MNISTModel()

# most basic trainer, uses good defaults (1 gpu)
trainer = pl.Trainer(gpus=1)    
trainer.fit(mnist_model)   

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:
  | Name | Type   | Params
----------------------------
0 | l1   | Linear | 7 K   


Extracting /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw/train-images-idx3-ubyte.gz to /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw


Extracting /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw/train-labels-idx1-ubyte.gz to /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw


Extracting /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw




Extracting /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/anderson/@python/pytorch-tutorial-anderson/05-pytorch-lighting/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:lightning:Detected KeyboardInterrupt, attempting graceful shutdown...


1